Открытие гугл диска

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
import os
import os.path
from PIL import Image
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import random
from torchvision import transforms
import math
import torch.nn as nn
import torch.nn.functional as F
import time

Дата лоадер

In [3]:
def get_pathes(base_folder ):
    paths = []
    for i in range(11):
        folder_name = f'{i:02d}'
        image_folder = os.path.join(base_folder, folder_name, 'image_0')
        movements_file = os.path.join(base_folder, folder_name, f'{folder_name}.txt')
        paths.append((image_folder,movements_file))
    return paths

def get_data(batch_size, base_folder = '/content/drive/MyDrive'):
    paths = get_pathes(base_folder)

    # Выбор папок для тренировочных, валидационных и тестовых данных
    validation_folder = paths[-2]  # Предпоследняя папка
    test_folder = paths[-1]        # Последняя папка
    train_folders = paths[:-2]     # Все остальные папки

    # Чтение движений из файла
    def create_movements(movements_file):
        movements = []
        with open(movements_file, 'r') as f:
            for line in f:
                data = list(map(float, line.strip().split()))
                movements.append(data)
        return movements

    # movements = create_movements(movements_file)

    # Преобразование движений
    # def transform_mov(movements):
    #     new_movements = []
    #     for matrix in movements:
    #         new_matrix = [
    #             matrix[0:4],
    #             matrix[4:8],
    #             matrix[8:12],
    #             [0, 0, 0, 1]
    #         ]
    #         new_matrix = torch.tensor(new_matrix, dtype=torch.float32)
    #         new_movements.append(new_matrix)

    #     res_movements = []
    #     for i in range(len(new_movements) - 1):
    #         T1_inv = torch.linalg.inv(new_movements[i])
    #         T_rel = torch.matmul(new_movements[i + 1], T1_inv)
    #         res_matrix = T_rel[:3]
    #         res_matrix = torch.cat([res_matrix[0], res_matrix[1], res_matrix[2]])
    #         res_movements.append(res_matrix)


    #     return res_movements

    # def transform_mov(movements):
    #     res_movements = []
    #     for i in range(len(movements)-1):
    #         m1 = torch.tensor(movements[i])
    #         m2 = torch.tensor(movements[i+1])
    #         res = m2-m1
    #         res_movements.append(res)
    #     return res_movements

    def transform_mov(movements):
        res_movements = []
        steps = [1, 2]
        for step in steps:
            for i in range(len(movements)-step):
                m1 = torch.tensor(movements[i])
                m2 = torch.tensor(movements[i + step])
                res = m2 - m1
                distance = math.sqrt(pow(res[3],2) + pow(res[7],2) + pow(res[11],2))
                res_movements.append([distance ])
        return res_movements

    def list_pair_images(image_folder):
        res_images = []
        all_image_files = sorted([os.path.join(image_folder, file) for file in os.listdir(image_folder) if file.endswith('.png')])
        steps = [1, 2]
        for step in steps:
            for i in range(len(all_image_files) - step):
                image1_path = all_image_files[i]
                image2_path = all_image_files[i + step]
                res_images.append( (image1_path, image2_path) )

        return res_images

    # movements = transform_mov(movements)

    # Оптимизированный класс Dataset
    class ImageDataset(Dataset):
        def __init__(self, image_folder, movements, transform= None ):
            # Получаем список всех файлов .png и сортируем их
            all_image_files = sorted([os.path.join(image_folder, file) for file in os.listdir(image_folder) if file.endswith('.png')])
            self.image_files = all_image_files
            self.movements = movements
            self.transform = transform
            self.list_images = list_pair_images(image_folder)

        def __len__(self):
            # Возвращаем минимальную длину между движениями и изображениями - 1
            # return min(len(self.movements), len(self.image_files) - 1)
            return  len(self.movements)


        def __getitem__(self, idx):
            # image1_path = self.image_files[idx]
            # image2_path = self.image_files[idx + 1]

            image1_path, image2_path = self.list_images[idx]

            image1 = Image.open(image1_path).convert('L')
            image2 = Image.open(image2_path).convert('L')

            x = 100
            y = 100
            image1_crop = image1.crop( (x, y, x + 320, y + 180) )
            image2_crop = image1.crop( (x, y, x + 320, y + 180) )

            if self.transform:
                image1 = self.transform(image1_crop)
                image2 = self.transform(image2_crop)

            # movement = self.movements[idx].clone().detach().float()if isinstance(self.movements[idx], torch.Tensor) else torch.tensor(self.movements[idx], dtype=torch.float32)
            movement = torch.tensor(self.movements[idx], dtype=torch.float32)
            return (image1, image2), movement


    # Трансформации для уменьшения использования памяти
    transform = transforms.Compose([
        # transforms.Grayscale(),
        # transforms.Resize((376,1240)),
        # transforms.CenterCrop((370,1226)),
        # transforms.Resize((180,320)),
        transforms.ToTensor()
    ])

    def load_dataset(folders):
        images, movements = [], []
        for image_folder, movements_file in folders:
            mov = create_movements(movements_file)
            mov = transform_mov(mov)
            movements.append(mov)
            images.append(image_folder)
        return images, movements

    train_images, train_movements = load_dataset(train_folders)
    train_dataset = []
    for index,image_folder in enumerate(train_images):
        dataset = ImageDataset(image_folder, train_movements[index], transform= transform)
        train_dataset.append(dataset)
    train_dataset = torch.utils.data.ConcatDataset(train_dataset)

    # arr_gist = np.zeros(100)
    # for arr_mov in train_movements:
    #     for mov in arr_mov:
    #       arr_gist[int(10 * mov[0])] += 1
    # h = 0.1
    # for x in arr_gist:
    #     print(h, '   ', x)
    #     h += 0.1

    # Загрузка валидационных данных
    val_images, val_movements = load_dataset([validation_folder])
    val_dataset = ImageDataset(val_images[0], val_movements[0], transform=transform)


   # Загрузка тестовых данных
    test_images, test_movements = load_dataset([test_folder])
    test_dataset = ImageDataset(test_images[0], test_movements[0], transform=transform)

    # DataLoaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle = True, num_workers=4, pin_memory=True)
    validation_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle = False, num_workers=4, pin_memory=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle = False, num_workers=4, pin_memory=True)

    return train_loader, validation_loader, test_loader



EVALUATION

In [4]:
def set_all_seeds(seed):
    os.environ["PL_GLOBAL_SEED"] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

def compute_accuracy(model, data_loader, device):
    model.eval()
    total_loss = 0.0
    num_batches = 0
    correct_predictions = 0   # Счетчик корректных предсказаний
    total_samples = 0         # Общее количество элементов

    criterion = torch.nn.MSELoss()

    with torch.no_grad():
        for batch_idx, ((feature1, feature2),targets) in enumerate(data_loader):
            feature1 = feature1.to(device)
            feature2 = feature2.to(device)
            targets = targets.to(device)

            predicted_y = model(feature1, feature2)

            # Вычисление ошибки
            print(predicted_y[0].item(), '   ', targets[0].item())
            # print(predicted_y, '      ', targets)
            # print(predicted_y[0].item(), predicted_y[1].item(), predicted_y[2].item(), predicted_y[3].item(), predicted_y[4].item(), predicted_y[5].item(), predicted_y[6].item(), predicted_y[7].item())
            # print(targets[0].item(), targets[1].item(), targets[2].item(), targets[3].item(), targets[4].item(), targets[5].item(), targets[6].item(), targets[7].item())
            loss = criterion(predicted_y, targets)
            total_loss += loss.item()
            num_batches += 1

            # Проверка критерия качества
            relative_difference = torch.abs((predicted_y - targets) / targets)  # Расчет относительного отклонения
            valid_predictions = torch.all(relative_difference <= 0.1, dim=1)    # Условие: отклонение <= 10%
            correct_predictions += torch.sum(valid_predictions).item()
            total_samples = total_samples + targets.size(0)


    avg_loss = total_loss / num_batches             # Расчет средней ошибки
    accuracy = correct_predictions / total_samples  # Расчет доли верных предсказаний
    return avg_loss, accuracy

Model ( EfficientNetV2 version 4 )

In [5]:
class MBConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, expansion_factor, stride, kernel_size= 3):
        super(MBConvBlock, self).__init__()

        self.use_residual = (in_channels == out_channels) and (stride == 1)
        hidden_dim = in_channels * expansion_factor

        self.conv = nn.Sequential(
            # Слой расширения (expansion)
            nn.Conv2d(in_channels, hidden_dim, kernel_size=1),
            # nn.BatchNorm2d(hidden_dim),
            nn.ReLU6(inplace=True),

            # Depthwise свертка
            nn.Conv2d(hidden_dim, hidden_dim, kernel_size=kernel_size, stride=stride,
                      padding=kernel_size // 2, groups=hidden_dim),
            # nn.BatchNorm2d(hidden_dim),
            nn.ReLU6(inplace=True),

            # Сжатие (projection)
            nn.Conv2d(hidden_dim, out_channels, kernel_size=1),
            # nn.BatchNorm2d(out_channels),

        )

    def forward(self, x):
        if self.use_residual:
            return x + self.conv(x)
            print(self.conv(x).shape)
        else:
            return self.conv(x)
            print(self.conv(x).shape)
'''
class Network(nn.Module):
    def __init__(self, num_classes=1):
        super(Network, self).__init__()

        # Первый сверточный слой с измененными параметрами
        self.conv_stem = nn.Conv2d(2, 32, kernel_size=(6, 10), stride=(2, 3), padding=1)# 184, 407
        self.bn1 = nn.BatchNorm2d(32)
        self.relu1 = nn.ReLU6(inplace=True)

        self.blocks = nn.Sequential(
            # MBConvBlock(32, 64, expansion_factor=1, stride=2), #
            MBConvBlock(32, 48, expansion_factor=6, stride=2),# 92, 204
            MBConvBlock(48, 64, expansion_factor=6, stride=2),# 46, 102
            MBConvBlock(64, 64, expansion_factor=6, stride=1), # 45, 100
            MBConvBlock(64, 128, expansion_factor=6, stride=2),# 23, 50
            MBConvBlock(128, 128, expansion_factor=6, stride=1),# 22, 45
            MBConvBlock(128, 256, expansion_factor=6, stride=2),# 11, 24

            # MBConvBlock(256, 256, expansion_factor=6, stride=1),#
            nn.Conv2d(256, 256, kernel_size= 4, stride= 2, groups= 256), # 6, 13
            nn.Conv2d(256, 256, kernel_size= 1, stride= 1), # 6, 13
            nn.BatchNorm2d(256),
            nn.ReLU6(inplace=True),
            nn.Dropout2d(p=0.2),

            MBConvBlock(256, 512, expansion_factor=6, stride=2), # 3, 7

            # MBConvBlock(512, 512, expansion_factor=6, stride=2) #
            nn.Conv2d(512, 512, kernel_size= 3, stride= 1, groups= 512), # 3, 7
            nn.Conv2d(512, 512, kernel_size= 1, stride= 1), # 3, 7
            nn.BatchNorm2d(512),
            nn.ReLU6(inplace=True),
            nn.Dropout2d(p=0.2)

        )

        # # Глобальный Average Pooling
        # self.global_pool = nn.AdaptiveAvgPool2d(1)

        # Полносвязный слой для классификации
        self.fc1 = nn.Linear(512 * 4, 16)
        self.relu_fc = nn.ReLU(inplace=True)
        self.dropout_fc = nn.Dropout(p= 0.4) # Dropout после первого FC слоя
        self.fc2 = nn.Linear(16, 1)

        self._initialize_weights()

    def forward(self, image1, image2):
        x = torch.cat((image1, image2), dim=1)
        x = self.relu1(self.bn1(self.conv_stem(x)))
        x = self.blocks(x)
        # x = self.relu2(self.bn2(self.conv_head(x)))
        # x = self.global_pool(x)
        x = torch.flatten(x, 1)
        x = self.dropout_fc(self.relu_fc(self.fc1(x)))
        # x = self.relu_fc(self.fc1(x))
        x = self.fc2(x)
        return x

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out')
                if m.bias is not None:
                    nn.init.zeros_(m.bias)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.ones_(m.weight)
                nn.init.zeros_(m.bias)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.zeros_(m.bias)

'''

class Network(nn.Module):
    def __init__(self, num_classes=1):
        super(Network, self).__init__()

        # Первый сверточный слой с измененными параметрами  На входе 180, 320
        self.conv_stem = nn.Conv2d(2, 32, kernel_size= 5, stride= 1, padding=1)#
        self.bn1 = nn.BatchNorm2d(32)
        self.relu1 = nn.ReLU6(inplace=True)

        self.blocks = nn.Sequential(
            # MBConvBlock(32, 64, expansion_factor=1, stride=2), #
            MBConvBlock(32, 48, expansion_factor=6, stride=2),# 92, 204
            MBConvBlock(48, 64, expansion_factor=6, stride=2),# 46, 102
            MBConvBlock(64, 64, expansion_factor=6, stride=1), # 45, 100
            MBConvBlock(64, 128, expansion_factor=6, stride=2),# 23, 50
            MBConvBlock(128, 128, expansion_factor=6, stride=1),# 22, 45
            MBConvBlock(128, 256, expansion_factor=6, stride=2),# 11, 24

            # MBConvBlock(256, 256, expansion_factor=6, stride=1),#
            nn.Conv2d(256, 256, kernel_size= 4, stride= 2, groups= 256), # 6, 13
            nn.Conv2d(256, 256, kernel_size= 1, stride= 1), # 6, 13
            # nn.BatchNorm2d(256),
            nn.ReLU6(inplace=True),
            # nn.Dropout2d(p=0.2),

            MBConvBlock(256, 512, expansion_factor=6, stride=2), # 3, 7

            # MBConvBlock(512, 512, expansion_factor=6, stride=2) #
            nn.Conv2d(512, 512, kernel_size= 3, stride= 1, groups= 512), # 3, 7
            nn.Conv2d(512, 512, kernel_size= 1, stride= 1), # 3, 7
            # nn.BatchNorm2d(512),
            nn.ReLU6(inplace=True),
            # nn.Dropout2d(p=0.2)

        )

        # # Глобальный Average Pooling
        # self.global_pool = nn.AdaptiveAvgPool2d(1)

        # Полносвязный слой для классификации
        self.fc1 = nn.Linear(512 * 3, 16)
        self.relu_fc = nn.ReLU(inplace=True)
        self.dropout_fc = nn.Dropout(p= 0.2) # Dropout после первого FC слоя
        self.fc2 = nn.Linear(16, 1)

        self._initialize_weights()

    def forward(self, image1, image2):
        x = torch.cat((image1, image2), dim=1)
        x = self.relu1(self.conv_stem(x))
        x = self.blocks(x)
        # x = self.relu2(self.bn2(self.conv_head(x)))
        # x = self.global_pool(x)
        x = torch.flatten(x, 1)
        x = self.dropout_fc(self.relu_fc(self.fc1(x)))
        # x = self.relu_fc(self.fc1(x))
        x = self.fc2(x)
        return x

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out')
                if m.bias is not None:
                    nn.init.zeros_(m.bias)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.ones_(m.weight)
                nn.init.zeros_(m.bias)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.zeros_(m.bias)

Model ( EfficientNetV2 version 5 )

In [ ]:

class SpatialAttention(nn.Module):
    def __init__(self, kernel_size=7):
        super(SpatialAttention, self).__init__()

        assert kernel_size in (3, 7), 'kernel size must be 3 or 7'
        padding = 3 if kernel_size == 7 else 1

        self.conv = nn.Conv2d(2, 1, kernel_size, padding=padding, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        input_x = x
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        x = torch.cat([avg_out, max_out], dim=1)
        x = self.conv(x)
        return self.sigmoid(x) * input_x


class Block(nn.Module):
    def __init__(self, in_channals, out_channals, kernel_size, stride):
        super(Block, self).__init__()
        self.conv_1 = nn.Conv2d(in_channals, out_channals, kernel_size= kernel_size, stride= 1)
        self.conv_2 = nn.Conv2d(out_channals, out_channals, kernel_size= kernel_size,
                                stride= stride, groups= out_channals)
        self.conv_3 =  nn.Conv2d(out_channals, out_channals, kernel_size= 1, stride= 1)
        self.relu6 = nn.ReLU6(inplace=True)

    def forvard(self, x):
        x = self.conv_1(x)
        x = self.conv_2(x)
        x = self.conv_3(x)
        x = self.relu6(x)

        return x


class Network(nn.Module):
    def __init__(self, num_classes=1):
        super(Network, self).__init__()

        # Первый сверточный слой с измененными параметрами  input image 180 X 320
        self.conv_stem = nn.Conv2d(2, 32, kernel_size= 20, stride= 1)# 161, 301
        self.bn1 = nn.BatchNorm2d(32)
        self.relu_1 = nn.ReLU6(inplace=True)
        self.spatial_attention = SpatialAttention(kernel_size=7)



        self.blocks1 = nn.Sequential(
            nn.Conv2d(32, 48, kernel_size= 10, stride= 1),  # 152, 292
            nn.Conv2d(48, 48, kernel_size= 10, stride= 2, groups= 48), # 72, 142
            nn.Conv2d(48, 48, kernel_size= 1, stride= 1), # 72, 142
            # nn.BatchNorm2d(64),
            nn.ReLU6(inplace=True),
            # nn.Dropout2d(p=0.2),
        )

        self.conv_1 = nn.Conv2d(48, 64, kernel_size= 10, stride= 1) # 63, 123

        self.blocks2 = nn.Sequential(
            nn.Conv2d(64, 96, kernel_size= 5, stride= 1), # 59, 119
            nn.Conv2d(96, 96, kernel_size= 5, stride= 2, groups= 96), # 28, 58
            nn.Conv2d(96, 96, kernel_size= 1, stride= 1), # 28, 58
            # nn.BatchNorm2d(128),
            nn.ReLU6(inplace=True),
            # nn.Dropout2d(p=0.2),
        )

        self.conv_2 = nn.Conv2d(96, 128, kernel_size= 3, stride= 1) # 26, 56

        self.blocks3 = nn.Sequential(
            nn.Conv2d(128, 196, kernel_size= 3, stride= 1), # 24, 54
            nn.Conv2d(196, 196, kernel_size= 3, stride= 2, groups= 196), # 11, 26
            nn.Conv2d(196, 196, kernel_size= 1, stride= 1), # 11, 26
            # nn.BatchNorm2d(256),
            nn.ReLU6(inplace=True),
            nn.Dropout2d(p=0.2),
        )

        self.conv_3 = nn.Conv2d(196, 256, kernel_size= 3, stride= 1) # 9, 24

        self.blocks4 = nn.Sequential(
            nn.Conv2d(256, 384, kernel_size= 3, stride= 1), # 7, 22
            nn.Conv2d(384, 384, kernel_size= 3, stride= 2, groups= 384), # 3, 10
            nn.Conv2d(384, 384, kernel_size= 1, stride= 1), # 3, 10
            # nn.BatchNorm2d(512),
            nn.ReLU6(inplace=True),
            # nn.Dropout2d(p=0.2),
        )

        self.conv_4 = nn.Conv2d(384, 512, kernel_size= 3, stride= 1) # 1, 8



        # # Глобальный Average Pooling
        # self.global_pool = nn.AdaptiveAvgPool2d(1)

        # Полносвязный слой для классификации
        self.fc1 = nn.Linear(512 * 10, 32)
        self.relu_fc = nn.ReLU(inplace=True)
        self.dropout_fc = nn.Dropout(p= 0.2) # Dropout после первого FC слоя
        self.fc2 = nn.Linear(32, 1)

        self._initialize_weights()

    def forward(self, image1, image2):
        x = torch.cat((image1, image2), dim=1)
        # x = self.relu1(self.bn1(self.conv_stem(x)))
        x = self.relu_1(self.conv_stem(x))
        # x = self.spatial_attention(x)
        x = self.blocks1(x)
        x = self.relu_1(self.conv_1(x))
        x = self.blocks2(x)
        x = self.relu_1(self.conv_2(x))
        x = self.blocks3(x)
        x = self.relu_1(self.conv_3(x))
        x = self.blocks4(x)
        x = self.relu_1(self.conv_4(x))
        # x = self.spatial_attention(x)
        # x = self.blocks4(x)
        # x = self.relu2(self.bn2(self.conv_head(x)))
        # x = self.global_pool(x)
        x = torch.flatten(x, 1)
        x = self.dropout_fc(self.relu_fc(self.fc1(x)))
        # x = self.relu_fc(self.fc1(x))
        x = self.fc2(x)
        return x

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out')
                if m.bias is not None:
                    nn.init.zeros_(m.bias)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.ones_(m.weight)
                nn.init.zeros_(m.bias)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.zeros_(m.bias)

Train

Модель FC


In [ ]:

class Network(nn.Module):
    def __init__(self, num_classes=1):
        super(Network, self).__init__()

        self.lin_1 = nn.Linear(180 * 320 * 2, 128)
        self.relu_fc = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
        self.lin_2 = nn.Linear(128, 16)
        self.lin_3 = nn.Linear(16, 1)
        self.dropout_fc = nn.Dropout(p= 0.2)


    def forward(self, image1, image2):
        # x = torch.cat((image1, image2), dim=1)
        im1 = torch.flatten(image1, 1)
        im2 = torch.flatten(image1, 1)
        x = torch.cat((im1, im2), dim= 1)
        x = self.lin_1(x)
        # x = self.relu_fc(x)
        x = self.sigmoid(x)
        x = self.lin_2(x)
        # x = self.relu_fc(x)
        x = self.sigmoid(x)
        x = self.dropout_fc(x)
        x = self.lin_3(x)

        return x

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.zeros_(m.bias)


In [18]:
def train(model, num_epochs, last_epoch, train_loader,
                validation_loader, test_loader,optimizer, device,scheduler,
          checkpoint_dict):
    start_time = time.time()
    minibatch_loss_list, train_loss_list, valid_loss_list = [] ,[], []
    criterion = torch.nn.MSELoss()

    for epoch in range(last_epoch, num_epochs):
        loss_train_total = 0
        num = 0
        model.train()
        for batch_idx, ((feature1, feature2), targets) in enumerate(train_loader):
            feature1 = feature1.to(device)
            feature2 = feature2.to(device)
            targets = targets.to(device)

            predicted_y = model(feature1,feature2)
            loss = criterion(predicted_y, targets)
            # loss =  torch.mean(targets - predicted_y)

            # loss_train_total += loss.item()
            loss_train_total += abs(loss.item())
            num += 1

            optimizer.zero_grad()

            loss.backward()
            optimizer.step()

            minibatch_loss_list.append(loss.item())
            if batch_idx %2000 == 0:
                print(f'Epoch: {epoch+1:03d}/{num_epochs:03d} '
                      f'| Batch {batch_idx:04d}/{len(train_loader):04d} '
                      f'| Loss: {loss.item():.4f}'
                      f'| Learning rate: {optimizer.param_groups[0]["lr"]}')
                print(predicted_y[0].item(), '   ', targets[0].item())
                model.eval()
                avg_loss_val, accuracy_val = compute_accuracy(model, validation_loader, device)
                print(f'Epoch: {epoch+1:03d}/{num_epochs:03d} '
                    f'| Train AVG LOSS: {loss_train_total / num: .4f}  \n'
                    # f'| Train AVG LOSS: {avg_loss_train: .4f} | Accuracy_train: {accuracy_train: .4f} \n'
                    f'| Validation AVG LOSS: {avg_loss_val: .4f} | Accuracy_val: {accuracy_val: .4f} ')
                model.train()
                # print(predicted_y[0].item(), predicted_y[1].item(), predicted_y[2].item(), predicted_y[3].item(), predicted_y[4].item(), predicted_y[5].item(), predicted_y[6].item(), predicted_y[7].item())
                # print(targets[0].item(), targets[1].item(), targets[2].item(), targets[3].item(), targets[4].item(), targets[5].item(), targets[6].item(), targets[7].item())
        # scheduler.step()
        model.eval()
        with torch.no_grad():
            avg_loss_train, accuracy_train = compute_accuracy(model, train_loader, device)
            # avg_loss_val, accuracy_val = compute_accuracy(model, validation_loader, device)
            # scheduler.step( avg_loss_val)
            print(f'Epoch: {epoch+1:03d}/{num_epochs:03d} '
                  f'| Train AVG LOSS: {loss_train_total / num: .4f}  \n'
                  # f'| Train AVG LOSS: {avg_loss_train: .4f} | Accuracy_train: {accuracy_train: .4f} \n'
                  f'| Validation AVG LOSS: {avg_loss_val: .4f} | Accuracy_val: {accuracy_val: .4f} ')
            # train_loss_list.append(avg_loss_train)
            valid_loss_list.append(avg_loss_val)

        model.train()
        scheduler.step()

        # checkpoint = {
        #     'model_state_dict': model.state_dict(),
        #     'optimizer_state_dict': optimizer.state_dict()
        # }

        checkpoint_dict['state_model'] = model.state_dict()
        checkpoint_dict['state_opt'] =  optimizer.state_dict()
        checkpoint_dict[ 'state_scheduler'] = scheduler.state_dict()
        # checkpoint_dict['train_loss'] = avg_loss_train
        checkpoint_dict['val_loss'] = avg_loss_val
        # checkpoint_dict['train_acc'] = accuracy_train
        checkpoint_dict['val_acc'] =  accuracy_val
        checkpoint_dict['EPOCHS'] = num_epochs
        checkpoint_dict['current_epoch'] = epoch + 1
        checkpoint_dict[ 'learning'] = optimizer.param_groups[0]["lr"]

        path = directory_path + str(epoch + 1) + '_epoch.pth'
        torch.save(checkpoint_dict, path)
        # path = '/content/drive/My Drive/checkpoint_model_3.pth'
        # torch.save(checkpoint, path)

        elapsed = (time.time() - start_time)/60
        print(f'Time elapsed: {elapsed:.2f} min')

    elapsed = (time.time() - start_time)/60
    print(f'Total Training Time: {elapsed:.2f} min')

    avg_loss_test, accuracy_test = compute_accuracy(model, test_loader,device)
    print(f'Test AVG LOSS: {avg_loss_test: .4f} | Accuracy_test: {accuracy_test: .4f}')

    return minibatch_loss_list, train_loss_list, valid_loss_list


Описание модели, оптимизатора, шедулера

In [7]:
txt_model_opt_shed = '''

class Network(nn.Module):
    def __init__(self, num_classes=1):
        super(Network, self).__init__()

        self.lin_1 = nn.Linear(180 * 320 * 2, 128)
        self.relu_fc = nn.ReLU()
        self.lin_2 = nn.Linear(128, 16)
        self.lin_3 = nn.Linear(16, 1)
        self.dropout_fc = nn.Dropout(p= 0.2)


    def forward(self, image1, image2):
        # x = torch.cat((image1, image2), dim=1)
        im1 = torch.flatten(image1, 1)
        im2 = torch.flatten(image1, 1)
        x = torch.cat((im1, im2), dim= 1)
        x = self.lin_1(x)
        x = self.relu_fc(x)
        x = self.lin_2(x)
        x = self.relu_fc(x)
        x = self.dropout_fc(x)
        x = self.lin_3(x)

        return x

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.zeros_(m.bias)


optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001, weight_decay = 0.002)

'''

Создание библиотеки для хранения сохранения

In [8]:
directory_path = '/content/drive/My Drive/checpoint/model_5_0/'

# Проверка существования директории
if not os.path.exists(directory_path):
    # Создание директории
    os.makedirs(directory_path)
    print(f"Директория {directory_path} была создана.")
else:
    print(f"Директория {directory_path} уже существует.")

Директория /content/drive/My Drive/checpoint/model_5_0/ уже существует.


Создание словаря для сохранения результатов

In [9]:
checkpoint_dict = {
    'model_description' : txt_model_opt_shed,
    'state_model' : None,
    'state_opt'   : None,
    'state_scheduler' : None,

    'train_loss' :None,
    'val_loss'   : None,
    'best_loss'  : None,
    'train_acc'  : None,
    'val_acc'    : None,

    'EPOCHS'     : None,
    'current_epoch' : None,
    'learning' : None
}

MAIN

In [10]:
BATCH_SIZE = 1
RANDOM_SEED = 123
NUM_EPOCHS = 20
last_epoch = 0
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Використовується пристрій: {DEVICE}")

Використовується пристрій: cuda


In [11]:
from torch.optim.lr_scheduler import StepLR
set_all_seeds(RANDOM_SEED)

model = Network()
model = model.to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr= 0.0003, weight_decay= 0.001)
# optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)
train_loader, validation_loader, test_loader = get_data(batch_size = BATCH_SIZE)
scheduler = StepLR(optimizer, step_size = 4, gamma = 0.3)
# scheduler = None
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 2,254,609 trainable parameters


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [ ]:
# Загрузка словаря из файла
path = './3_epoch.pth'
checkpoint = torch.load(path)
if scheduler == None:
    scheduler = StepLR(optimizer, step_size = 1, gamma = 1)
# # Загрузка состояния модели
# save_check = checkpoint['optimizer_state_dict']
# save_check['lr'] = 0.0003
model.load_state_dict( checkpoint['state_model'] )
optimizer.load_state_dict( checkpoint['state_opt'] )
scheduler.load_state_dict( checkpoint['state_scheduler'] )
last_epoch = checkpoint['current_epoch']
# # Загрузка состояния оптимизатора
# optimizer.load_state_dict(save_check)

In [20]:
if scheduler == None:
    scheduler = StepLR(optimizer, step_size = 1, gamma = 1)
minibatch_loss, train_acc, valid_acc = train(model = model,
            num_epochs = NUM_EPOCHS,
            last_epoch = last_epoch,
            train_loader = train_loader,
            validation_loader = validation_loader,
            test_loader = test_loader,
            optimizer = optimizer,
            device = DEVICE,
            scheduler = scheduler,
            checkpoint_dict = checkpoint_dict)

Epoch: 001/020 | Batch 0000/40791 | Loss: 0.1536| Learning rate: 0.0003
0.0006953999982215464     0.3926427662372589
0.0009801366832107306     0.2889880836009979
0.0009801366832107306     0.29325538873672485
0.0009801366832107306     0.29893848299980164
0.0009801366832107306     0.3062071204185486
0.0009801366832107306     0.3157193660736084
0.0009801367996260524     0.32351788878440857
0.0009801366832107306     0.33798491954803467
0.0009801366832107306     0.3556867837905884
0.0009801366832107306     0.3602379560470581
0.0009801366832107306     0.3732980787754059
0.0009801366832107306     0.3857421875
0.0009801366832107306     0.3946792185306549
0.0009801366832107306     0.4027072787284851
0.0009801366832107306     0.41747573018074036
0.0009801366832107306     0.4288489818572998
0.0009801366832107306     0.4427962005138397
0.0009801366832107306     0.45316970348358154
0.0009801366832107306     0.4688604176044464
0.0009801366832107306     0.48217636346817017
0.0009801366832107306     0

KeyboardInterrupt: 